# Ola Hallengrens CommandLog Table

This is where the output of Ola's jobs are stored if you choose `@LogToTable= 'Y'`

## Showing when commands ran
Which commands ran when in the last 7 days ?


In [19]:
SELECT 
      [DatabaseName]
      ,[SchemaName]
      ,[ObjectName]
      ,[CommandType]
      ,[StartTime]
      ,[EndTime]
      ,[ErrorNumber]
      ,[ErrorMessage]
  FROM [dbo].[CommandLog]
  WHERE StartTime > DATEADD(DAY,-7, GETDATE())
  ORDER BY StartTime DESC

## Getting Failures

Get the failures in teh last 7 days

In [24]:
SELECT 
      [DatabaseName]
      ,[SchemaName]
      ,[ObjectName]
      ,[CommandType]
      ,[StartTime]
      ,[EndTime]
      ,[ErrorNumber]
      ,[ErrorMessage]
  FROM [dbo].[CommandLog]
  WHERE ErrorNumber <> 0
  AND StartTime > DATEADD(DAY,-7, GETDATE())
  ORDER BY StartTime DESC

## Index fragmentation Duration
Getting the Last 7 days of Ola Hallengrens Index Alter commands duration and fragmentation

https://jbswiki.com/2019/02/27/tsql-to-query-commandlog-table-to-check-alter-index-duration/

In [5]:
USE MASTER
GO
SELECT
[DatabaseName]
      ,[SchemaName]
      ,[ObjectName]
      ,[ObjectType]
      ,[IndexName]
      ,[IndexType]
      ,[CommandType]
         ,ExtendedInfo.value('(/ExtendedInfo/PageCount)[1]', 'int') as Page_Count,convert(decimal(5,2),ExtendedInfo.value('(/ExtendedInfo/Fragmentation)[1]', 'Float')) as Fragmentation
       ,CASE WHEN ([Command] LIKE '%REBUILD%') THEN 'REBUILD' ELSE 'REORGANIZE' END AS [TYPE]
       , CAST(DATEDIFF(second, StartTime, EndTime) / 60 / 60 / 24 / 7 AS NVARCHAR(50)) + ':'
    + CAST(DATEDIFF(second, StartTime, EndTime) / 60 / 60 / 24 % 7 AS NVARCHAR(50)) + ':'
    + CAST(DATEDIFF(second, StartTime, EndTime) / 60 / 60 % 24  AS NVARCHAR(50)) + ':'
    + CAST(DATEDIFF(second, StartTime, EndTime) / 60 % 60 AS NVARCHAR(50)) + ':'
    + CAST(DATEDIFF(second, StartTime, EndTime) % 60 AS NVARCHAR(50)) + '' as "w:d:h:m:s"
       , DATEDIFF(second, StartTime, EndTime) [Duration in Sec]
      ,[StartTime]
      ,[EndTime]
              from [dbo].[Commandlog]
WHERE CommandType = 'ALTER_INDEX'
AND StartTime > DATEADD(DAY,-7, GETDATE())
order by DATEDIFF(second, StartTime, EndTime) desc

## How Many Times has an Index been Rebuilt or Re-Organised?
Get the number of times an index has been rebuilt or re-organised in the last 180 days  

https://gallery.technet.microsoft.com/scriptcenter/TSQL-TO-QUERY-COMMANDLOG-31458c62

In [6]:
SELECT TOP 100 [DatabaseName], [ObjectName], [IndexName], count(*) as [Frequency],
		CASE WHEN ([Command] LIKE '%REBUILD%') THEN 'REBUILD' ELSE 'REORGANIZE' END AS [TYPE]
FROM [dbo].[CommandLog]
WHERE [commandtype] = 'ALTER_INDEX' 
AND [starttime] > DATEADD(d,-180,GETDATE())
GROUP BY [DatabaseName], [ObjectName], [IndexName], [Command]
ORDER BY [Frequency] DESC

## CHECKDB Duration
Get the duration of the CHECKDB command for the last 7 days  
https://sqlnotesfromtheunderground.wordpress.com/2013/11/06/knowing-your-checkdb-duration-times-using-ola-hallengren-maintenance-scripts/

In [9]:
WITH    rankcte ( [ORDER], databasename, [duration] )
          AS ( SELECT   RANK() OVER ( PARTITION BY DatabaseName ORDER BY StartTime DESC ) AS 'order' ,
                        [DatabaseName] ,
                        DATEDIFF(SECOND, [StartTime], [EndTime]) AS 'duration'
               FROM     [dbo].[CommandLog]
               WHERE    Command LIKE '%DBCC CHECKDB%'
                        AND StartTime > GETDATE() - 180
             )
    SELECT  *
    FROM    rankcte PIVOT
( SUM([duration]) FOR [ORDER] IN ( [1], [2], [3], [4], [5], [6], [7] ) ) AS resultTable;

The following query will help you get information about failures and the error log the message is in   
http://www.cookingsql.com/2017/02/querying-commandlog/

In [25]:

SELECT
DatabaseName,
StartTime,
DATEDIFF(MINUTE,StartTime,EndTime) 'Duration in Min',
EndTime,
ErrorNumber,
ErrorMessage,
Command,
replace(
cast(SERVERPROPERTY('ErrorLogFileName') as nvarchar(1000))
,'errorlog','')
FROM DBO.COMMANDLOG
WHERE 1=1
--and StartTime > dateadd(dd,-1,getdate())
--and CommandType in ('BACKUP_DATABASE','BACKUP_LOG','RESTORE_VERIFYONLY')
--and CommandType in ('xp_create_subdir','xp_delete_file')
--and CommandType in ('DBCC_CHECKDB','ALTER_INDEX')

and ErrorNumber <> 0

order by ID desc